In [2]:
%%capture no-stderr
%pip install -qU  langchain-google-genai    langchain_community langchain langgraph

In [3]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [4]:
_set_env("GOOGLE_API_KEY")

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [6]:
# Sub process is used for running the shell commands in the python script
from langchain.tools import  tool
import subprocess

@tool
def github_commit_tool(commit_message : str = "Some Commit") -> str:
    """ This function uses the subprocess module to commit the changes to the github repository pre-defined"""
    subprocess.run(["git","add", "."])
    subprocess.run(["git","commit", "-m", str(commit_message)])
    subprocess.run(["git","push","-u","origin","main"]  , shell=True)

    return "Commited to Github"


In [ ]:
github_commit_tool("Initial Commit")

In [7]:
tools = [github_commit_tool]

In [8]:
from langgraph.graph import add_messages
from pydantic import BaseModel
from typing import Annotated

from langchain_core.messages import SystemMessage
class State(BaseModel):
    messages : Annotated[list , add_messages]

# System message
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing and helping users in making git and github related operations")

# LLM Node
def model(message : State): 
    return {"messages": [llm.invoke( [sys_msg] + message.messages ) ]}

In [9]:

from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from IPython.display import Image, display
from langgraph.graph.state import CompiledStateGraph
